In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [6]:
# 选择图片和标签数据
target = 3
data_name = ['0618', '0854', '1066'][target - 1]
img_path = f'../input_data/{data_name}.png'  # 原图路径
json_path = f'../input_data/{data_name}.json'  # Labelme 标注文件路径
output_features_filename = f'./Features/{data_name}_HSV1x1_features.npy'  # 数据输出路径
output_labels_filename = f'./Features/{data_name}_HSV1x1_labels.npy'  # 数据输出路径

In [7]:
# 加载原图
img = cv2.imread(img_path)

img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 读取 Labelme JSON 数据
with open(json_path) as f:
    label_data = json.load(f)

# 创建空的标签图S
height, width, _ = img.shape
label_img = np.zeros((height, width), dtype=np.uint8)

# 根据多边形标注生成标签图
for shape in label_data['shapes']:
    label = shape['label']
    points = np.array(shape['points'], dtype=np.int32)
    if label == 'road':
        cv2.fillPoly(label_img, [points], 255)  # 道路: road
    elif label == 'other':
        cv2.fillPoly(label_img, [points], 0)   # 其他: other 
    elif label == 'bg':
        cv2.fillPoly(label_img, [points], 128)    # 主背景: bg

# 确定采样步长
step_size = 10  # 增大步长以减少采样点数量

# 数据暂存
features = []
labels = []
sampling_points = []

# 进行均匀采样
for y in range(1, height - 1, step_size):
    for x in range(1, width - 1, step_size):
        feature_vector = img_hsv[y, x].flatten()  
        features.append(feature_vector)

        # 根据标签图确定标签
        label_value = label_img[y, x]
        if label_value == 255:  # road
            labels.append(1)
        elif label_value == 0:  # other
            labels.append(2)
        else:
            labels.append(0)  # background
        
        # 记录采样点位置
        sampling_points.append((x, y))

# 转换为NumPy数组
features_array = np.array(features)
labels_array = np.array(labels)

# 保存数据到npy文件
np.save(output_features_filename, features_array)
np.save(output_labels_filename, labels_array)

print(f"Features saved to {output_features_filename} \
      \n Labels saved to {output_labels_filename}.")


Features saved to ./Features/1066_HSV1x1_features.npy       
 Labels saved to ./Features/1066_HSV1x1_labels.npy.
